In [54]:
import pandas as pd

from pyspark.sql import SparkSession
from pyspark.sql.functions import *

import re

In [7]:
spark = SparkSession.builder.getOrCreate()

In [8]:
spark

1. Read the case, department, and source data into their own spark dataframes.

In [37]:
df = spark.read.csv('data/case.csv', header=True)

In [12]:
dept = spark.read.csv('data/dept.csv')

In [13]:
source = spark.read.csv('data/source.csv')

2. Let's see how writing to the local disk works in spark:


- Write the code necessary to store the source data in both csv and json format, store these as sources_csv and sources_json
- Inspect your folder structure. What do you notice?
    > """not saved with file extentions?"""

In [18]:
source.write.csv('data/sources_csv', mode = 'overwrite')

In [19]:
source.write.json('data/sources_json', mode = 'overwrite')

3. Inspect the data in your dataframes. Are the data types appropriate? Write the code necessary to cast the values to the appropriate types.

In [20]:
source.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)



In [21]:
dept.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c3: string (nullable = true)



In [38]:
df.printSchema()

root
 |-- case_id: string (nullable = true)
 |-- case_opened_date: string (nullable = true)
 |-- case_closed_date: string (nullable = true)
 |-- SLA_due_date: string (nullable = true)
 |-- case_late: string (nullable = true)
 |-- num_days_late: string (nullable = true)
 |-- case_closed: string (nullable = true)
 |-- dept_division: string (nullable = true)
 |-- service_request_type: string (nullable = true)
 |-- SLA_days: string (nullable = true)
 |-- case_status: string (nullable = true)
 |-- source_id: string (nullable = true)
 |-- request_address: string (nullable = true)
 |-- council_district: string (nullable = true)



In [39]:
df.show(1, vertical= True)

-RECORD 0------------------------------------
 case_id              | 1014127332           
 case_opened_date     | 1/1/18 0:42          
 case_closed_date     | 1/1/18 12:29         
 SLA_due_date         | 9/26/20 0:42         
 case_late            | NO                   
 num_days_late        | -998.5087616000001   
 case_closed          | YES                  
 dept_division        | Field Operations     
 service_request_type | Stray Animal         
 SLA_days             | 999.0                
 case_status          | Closed               
 source_id            | svcCRMLS             
 request_address      | 2315  EL PASO ST,... 
 council_district     | 5                    
only showing top 1 row



In [52]:
(
    df.withColumn(col('case_opened_date')).cast(datetime)
    .withColumn(col('case_closed_date')).cast(datetime)
    .withColumn(col('case_closed_date')).cast(datetime)
)


TypeError: withColumn() missing 1 required positional argument: 'col'

In [88]:
df.select(df.request_address).show(truncate=False)

+----------------------------------------+
|request_address                         |
+----------------------------------------+
|2315  EL PASO ST, San Antonio, 78207    |
|2215  GOLIAD RD, San Antonio, 78223     |
|102  PALFREY ST W, San Antonio, 78223   |
|114  LA GARDE ST, San Antonio, 78223    |
|734  CLEARVIEW DR, San Antonio, 78228   |
|BANDERA RD and BRESNAHAN                |
|10133  FIGARO CANYON, San Antonio, 78251|
|10133  FIGARO CANYON, San Antonio, 78251|
|10133  FIGARO CANYON, San Antonio, 78251|
|10133  FIGARO CANYON, San Antonio, 78251|
|10133  FIGARO CANYON, San Antonio, 78251|
|10133  FIGARO CANYON, San Antonio, 78251|
|10129  BOXING PASS, San Antonio, 78251  |
|10129  BOXING PASS, San Antonio, 78251  |
|10129  BOXING PASS, San Antonio, 78251  |
|834  BARREL POINT, San Antonio, 78251   |
|834  BARREL POINT, San Antonio, 78251   |
|834  BARREL POINT, San Antonio, 78251   |
|834  BARREL POINT, San Antonio, 78251   |
|834  BARREL POINT, San Antonio, 78251   |
+----------

In [97]:
df.filter(df.request_address == r'2035\s.*').show()

+-------+----------------+----------------+------------+---------+-------------+-----------+-------------+--------------------+--------+-----------+---------+---------------+----------------+
|case_id|case_opened_date|case_closed_date|SLA_due_date|case_late|num_days_late|case_closed|dept_division|service_request_type|SLA_days|case_status|source_id|request_address|council_district|
+-------+----------------+----------------+------------+---------+-------------+-----------+-------------+--------------------+--------+-----------+---------+---------------+----------------+
+-------+----------------+----------------+------------+---------+-------------+-----------+-------------+--------------------+--------+-----------+---------+---------------+----------------+



In [96]:
re.findall(r'2035\s.*', df.select(df.request_address))

TypeError: expected string or bytes-like object